In [1]:
import numpy as np
from gameimage import board_to_image, update_image
import chess
from game import Game
from actionspace import map_w, map_b
import tensorflow as tf
from tf_funcs import predict_fn
from train.config import TrainingConfig
from math import log
from time import time
from trt_funcs import load_trt_model_latest
import pympler.asizeof
trt_func, model = load_trt_model_latest()


2024-03-22 19:05:00.209538: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
trt_func, model = load_trt_model_latest()


2024-03-22 19:05:01.742528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5937 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-03-22 19:05:03.265155: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [12]:
from mcts.c import run_mcts
from test_games import ruy_lopez, scholars_mate_cpp
from time import time
config = TrainingConfig()
config.num_mcts_sampling_moves = 0
game = Game(scholars_mate_cpp)
#game.board

In [22]:
t1 = time()
move, root, visit_counts = run_mcts(game, config, trt_func, 800, return_statistics=True)
t2 = time()
print(t2-t1)
print(move)

3.704056739807129
h5f7


In [7]:
from test_games import ruy_lopez, scholars_mate
from mcts2 import MCTS
config = TrainingConfig()
game = Game(scholars_mate)
mcts = MCTS(config, trt_func, False)
move, visit_counts = mcts(game, 800, verbose_move=True)

Time for MCTS: 4.034390926361084
Visits: 826
Summed value: 267.908492
Evaluation: 0.660830
Best variation: h5f7
Move          Visits  NN Output                    Policy             Prior          Avg. value     UCB            Q+U           
----------------------------------------------------------------------------------------------------------------------------------
h5f7  (None)  N: 293  (v: 1.000000, p:  0.224338)  (p_norm:   2.70%)  (P: 0.026956)  (Q: 1.000000)  (U: 1.013210)  (Q+U: 2.013210)
h5g4  (None)  N: 22   (v: 0.361728, p: -0.083441)  (p_norm:   2.05%)  (P: 0.018815)  (Q: 0.476211)  (U: 0.593670)  (Q+U: 1.069881)
h5h7  (None)  N: 20   (v: 0.346541, p: -0.216094)  (p_norm:   1.82%)  (P: 0.014767)  (Q: 0.494577)  (U: 0.595550)  (Q+U: 1.090127)
h5e5  (None)  N: 20   (v: 0.331553, p: -0.040489)  (p_norm:   2.13%)  (P: 0.018228)  (Q: 0.496994)  (U: 0.621632)  (Q+U: 1.118625)
c4a6  (None)  N: 20   (v: 0.354769, p: -0.116476)  (p_norm:   1.99%)  (P: 0.016684)  (Q: 0.486397)  (U:

In [4]:
move, child_visits = mcts(game, 800, minimal_exploration=False, return_statistics=True)
print(move, len(child_visits[child_visits > 0]))
print(pympler.asizeof.asizeof(mcts.root) / 1024 / 1024, "MB")

h2h3 43
7.62939453125e-05 MB


In [5]:
new_root = mcts.root[move]
del mcts.root

In [8]:
print(new_root)

In [8]:
game.make_move(move)
mcts.root = mcts.root[move]
move, child_visits = mcts(game, 800, minimal_exploration=False, return_statistics=True)
print(move, len(child_visits[child_visits > 0]))
print(pympler.asizeof.asizeof(mcts) / 1024 / 1024, "MB")

f6e4 29
0.00011444091796875 MB


In [8]:
mcts.root.reset()
print(pympler.asizeof.asizeof(mcts) / 1024 / 1024, "MB")

0.00011444091796875 MB


In [9]:
print(pympler.asizeof.asizeof(mcts.root) / 1024 / 1024, "MB")

7.62939453125e-05 MB


In [5]:
from test_games import ruy_lopez, scholars_mate
from mcts import run_mcts
config = TrainingConfig()
game = Game(scholars_mate)
move, _ = run_mcts(
        config,
        game,
        trt_func,
        None,
        False,
        800,
        0,
        False,
        config.pb_c_factor,
        verbose_move=True
    )

Time taken: 0.83s
Visits: 800
Summed value: 129.518767
Evaluation: 0.399608
Best variation: h5f7
Move          Visits  NN Output                    Policy             Prior          Avg. value     UCB            Q+U           
----------------------------------------------------------------------------------------------------------------------------------
h5f7  (4395)  N: 542  (v: 0.000000, p: -0.140139)  (p_norm:   2.03%)  (P: 0.020348)  (Q: 1.000000)  (U: 1.005381)  (Q+U: 2.005381)
h5g5  (4386)  N: 8    (v: 0.402944, p:  0.287488)  (p_norm:   2.98%)  (P: 0.029758)  (Q: 0.503448)  (U: 0.978217)  (Q+U: 1.481665)
h5h3  (4392)  N: 8    (v: 0.409176, p:  0.250946)  (p_norm:   2.88%)  (P: 0.028807)  (Q: 0.505347)  (U: 0.964942)  (Q+U: 1.470290)
c4f7  (1404)  N: 8    (v: 0.388092, p:  0.253911)  (p_norm:   2.89%)  (P: 0.028883)  (Q: 0.526062)  (U: 0.986871)  (Q+U: 1.512932)
c4b5  (1394)  N: 8    (v: 0.387412, p:  0.255791)  (p_norm:   2.89%)  (P: 0.028931)  (Q: 0.495043)  (U: 0.956622)  (Q+

In [26]:
from train.model import generate_model
from train.config import TrainingConfig
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.compiler.tensorrt import trt_convert as trt
config = TrainingConfig()

In [27]:
def input_fn():
    positions = os.listdir("train/conversion_data")
    images = []
    for pos in positions:
        data_np = np.load(f"train/conversion_data/{pos}")
        images.append(data_np["image"])
    images = np.array(images).astype(np.float32)
    images[:, -1] /= 99.0
    for _ in range(len(images) // config.num_parallel_reads):
        indices = np.random.choice(images.shape[0], config.num_parallel_reads, replace=False)
        batch = images[indices]
        yield batch



In [29]:
model = generate_model()
model.save("test")


2024-03-22 16:02:49.857853: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5659 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5


INFO:tensorflow:Assets written to: test/assets


INFO:tensorflow:Assets written to: test/assets


In [30]:
conversion_params = trt.TrtConversionParams(
    precision_mode=trt.TrtPrecisionMode.INT8,
    use_calibration=True,
)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir="test",
    conversion_params=conversion_params,
)

converter.convert(calibration_input_fn=input_fn)

converter.save("test")

INFO:tensorflow:Linked TensorRT version: (8, 4, 3)


INFO:tensorflow:Linked TensorRT version: (8, 4, 3)


INFO:tensorflow:Loaded TensorRT version: (8, 6, 1)


INFO:tensorflow:Loaded TensorRT version: (8, 6, 1)


INFO:tensorflow:Loaded TensorRT 8.6.1 and linked TensorFlow against TensorRT 8.4.3. This is supported because TensorRT minor/patch upgrades are backward compatible.


...

2024-03-22 16:05:52.122659: I tensorflow/compiler/tf2tensorrt/common/utils.cc:104] Linked TensorRT version: 8.4.3
2024-03-22 16:05:52.122759: I tensorflow/compiler/tf2tensorrt/common/utils.cc:106] Loaded TensorRT version: 8.6.1
2024-03-22 16:05:58.622132: I tensorflow/compiler/tf2tensorrt/convert/convert_nodes.cc:1330] [TF-TRT] Sparse compute capability: enabled.
2024-03-22 16:05:58.633115: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger BuilderFlag::kENABLE_TACTIC_HEURISTIC has been ignored in this builder run. This feature is only supported on Ampere and beyond.
2024-03-22 16:06:00.915738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


[ 48  87  25  81  72  47  40  92  35  13 116  76  28  15   5 106 101  80
  30  37 109  12 117   1 113  20  22   8  21  41  11 115]
[ 69  35  65  48 114  89  14  42  73 108  11  70  37 110  54  90  31 104
  77  36 112  50  39 113  79  49 111  63  34 100  25  61]
[ 91  38  11 107  67  14  95  80  59  57  28  41 103  29  71  33  78  75
  87  58  26  35  20  30  39  93 108   0  72 114 109 101]


2024-03-22 16:06:54.396658: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger TensorRT encountered issues when converting weights between types and that could affect accuracy.
2024-03-22 16:06:54.396686: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger Check verbose logs for the list of affected weights.
2024-03-22 16:06:54.396701: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger - 16 weights are affected by this issue: Detected subnormal FP16 values.
2024-03-22 16:06:54.396730: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger - 4 weights are affected by this issue: Detected values less than smallest positive FP16 subnormal value and converted them to the FP16 minimum subnormalized value.


INFO:tensorflow:Assets written to: test/assets


INFO:tensorflow:Assets written to: test/assets


In [1]:
import numpy as np
from gameimage import board_to_image, update_image
import chess
from game import Game
from actionspace import map_w, map_b
import tensorflow as tf
from tf_funcs import predict_fn
from train.config import TrainingConfig
from math import log
from time import time
from trt_funcs import load_trt_model_latest
import pympler.asizeof
trt_func, model = load_trt_model_latest()

2024-03-22 22:22:58.036614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 22:23:00.024323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5793 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-03-22 22:23:01.602258: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [2]:
def play_game(config, trt_func):
    game = Game(chess.Board())
    search_statistics, images, player_on_turn = [], [], []
    was_full_search = True
    root = None
    while not game.terminal_with_outcome():
        # Playout cap randomization
        do_full_search = True if np.random.default_rng().random() < config.playout_cap_random_p else False
        num_simulations = config.num_mcts_sims[1] if do_full_search else config.num_mcts_sims[0]

        # Run MCTS 
        move, root, statistics = run_mcts(
            game=game,
            config=config,
            trt_func=trt_func,
            num_simulations=num_simulations,
            minimal_exploration=not do_full_search,
            root=root,
            reuse_tree=not was_full_search and not do_full_search,
            return_statistics=do_full_search
        )
        
        # Only moves with full search depth are added to the buffer for training
        if do_full_search:
            search_statistics.append(statistics)
            images.append(root.image)
            player_on_turn.append(game.to_play())
            
        was_full_search = do_full_search
        root = root.children[move]
        game.make_move(move)
    summary = f"{game.outcome_str}. Full search moves: {len(images)}/{game.history_len}."
    # Get terminal values for game states based on whos turn it was
    terminal_values = [game.terminal_value(player) for player in player_on_turn]
    return images, (terminal_values, search_statistics), summary

In [4]:
from train.play_game import play_game

import objgraph 

config = TrainingConfig()
for i in range(5):
    images, (terminal_values, search_stats), summary = play_game(config, trt_func)
    print(summary)

objgraph.show_most_common_types()
objgraph.show_growth()

Draw: outcome=Threefold repetition. Full search moves: 106/399.
White wins: outcome=Checkmate. Full search moves: 13/51.
Black wins: outcome=Checkmate. Full search moves: 34/154.
Draw: outcome=Fifty moves. Full search moves: 97/392.
Draw: outcome=Insufficient material. Full search moves: 79/289.
function      64056
dict          44227
tuple         33911
Parameter     28276
cell          21427
list          13742
ReferenceType 10654
OrderedDict   6305
mappingproxy  6167
Signature     6110
list                          13741       +66
dict                          38729       +64
ReferenceType                 10654       +54
builtin_function_or_method     4665       +53
OrderedDict                    6305       +22
tuple                         33903       +18
Parameter                     28276       +16
set                            2286       +12
TensorShape                      53       +10
Tensor                          234       +10


In [1]:
import objgraph

ModuleNotFoundError: No module named 'objgraph'